# Tracking de objetos con YoloV8 y Bytetrack

In [1]:
# !pip install ultralytics==8.0.84
# !pip install Cython
# !pip install numpy
# !pip install lap


In [2]:
import os
# esto es para evitar un error en Windows: OMP: Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import cv2
import numpy as np
from ultralytics.nn.autobackend import AutoBackend
from ultralytics.yolo.utils.plotting import Annotator, colors
import torch
from bytetrack.byte_tracker import BYTETracker
from ultralytics.yolo.data.dataloaders.stream_loaders import LoadImages
from ultralytics.yolo.utils.ops import non_max_suppression, scale_boxes
import time
import cv2


In [3]:

print(torch.cuda.is_available())

True


In [4]:
conf_thres = 0.25
iou_thres = 0.45
classes = None
agnostic_nms = False
max_det = 1000
line_thickness = 2
half = False
imgsz = (1280, 704)
vid_stride = 1

In [5]:
save_vid = True
video_file = "../../stream_bitrate2.mp4"


In [6]:
# Verificar si CUDA (GPU) está disponible y luego seleccionar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cuda


In [7]:
model = AutoBackend("yolov8n.pt").to(device)
model.warmup()
stride, names, pt = model.stride, model.names, model.pt


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


In [8]:
bytetracker = BYTETracker(
    track_thresh=0.6, match_thresh=0.8, track_buffer=120, frame_rate=30
)
tracker = bytetracker

In [9]:
dataset = LoadImages(
    video_file,
    imgsz=imgsz,
    stride=stride,
    auto=pt,
    transforms=None,
    vid_stride=vid_stride,
)


In [10]:
zona_personas = (805, 483, 1101, 618)

tiempos_deteccion = {}

def interseccion_zona(bbox, zona):
    """ Verifica si la caja delimitadora (bbox) se cruza con la zona definida. """
    x1, y1, x2, y2 = bbox
    zx1, zy1, zx2, zy2 = zona
    return not (x2 < zx1 or x1 > zx2 or y2 < zy1 or y1 > zy2)

In [11]:
for frame_idx, batch in enumerate(dataset):
    tiempo_actual = time.time()
    path, im, im0s, vid_cap, s = batch
    detections = np.empty((0, 5))
    # Asegurarse de que los datos de entrada (imágenes) también estén en la GPU
    im = torch.from_numpy(im).to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255.0  # 0 - 255 to 0.0 - 1.0
    im = torch.unsqueeze(im, 0)

    result = model(im)

    p = non_max_suppression(
        result, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det
    )

    for i, det in enumerate(p):
        p, im0, _ = path, im0s.copy(), getattr(dataset, "frame", 0)

        if det is not None and len(det):
            det[:, :4] = scale_boxes(
                im.shape[2:], det[:, :4], im0.shape
            ).round()  # rescale boxes to im0 size

        track_result = tracker.update(det.cpu(), im0)

        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        # draw boxes for visualization
        if len(track_result) > 0:
            for j, (output) in enumerate(track_result):
                bbox = output[0:4]
                id = output[4]
                cls = output[5]
                conf = output[6]

                c = int(cls)  # integer class
                id = int(id)  # integer id
                #label = f"{id} {names[c]} {conf:.2f}"

                if id not in tiempos_deteccion:
                    tiempos_deteccion[id] = tiempo_actual

                tiempo_deteccion = tiempo_actual - tiempos_deteccion[id]

                c = int(cls)  # integer class
                label = f"{id} {names[c]} {tiempo_deteccion:.2f}s" 

                if interseccion_zona(bbox, zona_personas):
                    color_especial = (0, 255, 0)  # Color verde, por ejemplo
                    annotator.box_label(bbox, label, color=color_especial)
                else:
                    color = colors(c, True)
                    annotator.box_label(bbox, label, color=color)

                """ 
                color = colors(c, True)
                annotator.box_label(bbox, label, color=color)
                """
                

    # Stream results
    im0 = annotator.result()
    cv2.imshow(str(p), im0)
    if cv2.waitKey(20) == 27:  # Presiona 'Esc' para salir
        break

cv2.destroyAllWindows()

Guardar el video

In [12]:

save_vid = True
video_file = "../../stream_bitrate2.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec
fps = 24  # Asumiendo 30 fps, ajusta según tu video
save_path = '../../output.mp4'
vid_writer = cv2.VideoWriter(save_path, fourcc, fps, (1280, 704))  # Ajusta la resolución según sea necesario


for frame_idx, batch in enumerate(dataset):
    tiempo_actual = time.time()
    path, im, im0s, vid_cap, s = batch
    detections = np.empty((0, 5))
    # Asegurarse de que los datos de entrada (imágenes) también estén en la GPU
    im = torch.from_numpy(im).to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255.0  # 0 - 255 to 0.0 - 1.0
    im = torch.unsqueeze(im, 0)

    result = model(im)

    p = non_max_suppression(
        result, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det
    )

    for i, det in enumerate(p):
        p, im0, _ = path, im0s.copy(), getattr(dataset, "frame", 0)

        if det is not None and len(det):
            det[:, :4] = scale_boxes(
                im.shape[2:], det[:, :4], im0.shape
            ).round()  # rescale boxes to im0 size

        track_result = tracker.update(det.cpu(), im0)

        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        # draw boxes for visualization
        if len(track_result) > 0:
            for j, (output) in enumerate(track_result):
                bbox = output[0:4]
                id = output[4]
                cls = output[5]
                conf = output[6]

                c = int(cls)  # integer class
                id = int(id)  # integer id
                #label = f"{id} {names[c]} {conf:.2f}"

                if id not in tiempos_deteccion:
                    tiempos_deteccion[id] = tiempo_actual

                tiempo_deteccion = tiempo_actual - tiempos_deteccion[id]

                c = int(cls)  # integer class
                label = f"{id} {names[c]} {tiempo_deteccion:.2f}s" 

                if interseccion_zona(bbox, zona_personas):
                    color_especial = (0, 255, 0)  # Color verde, por ejemplo
                    annotator.box_label(bbox, label, color=color_especial)
                else:
                    color = colors(c, True)
                    annotator.box_label(bbox, label, color=color)

                """ 
                color = colors(c, True)
                annotator.box_label(bbox, label, color=color)
                """
                

    # Save results
    im0 = annotator.result()
    vid_writer.write(im0)



vid_writer.release()
cv2.destroyAllWindows()